# Movie Lens 1M Data

This demo is based on an example in the book: https://www.amazon.com/dp/1449319793/

_Python for Data Analysis_ by William Wesley McKinney (O'Reilly). Copyright 2012 William McKinney, 978-1-449-31979-3

The data is downloadable from http://grouplens.org/datasets/movielens/

## Setup

- Install Anaconda from https://www.continuum.io/downloads
- Set the account name, user name and password to the environment variables to ``SF_ACCOUNT``, ``SF_USER`` and ``SF_PASSWORD``
- Set the S3 Bucket, AWS access key id and AWS secret access key to ``SF_S3_BUCKET``, ``AWS_ACCESS_KEY_ID`` and ``AWS_SECRET_ACCESS_KEY`` to load the daata from the given location.
- Start Jupyter notebook
```jupyter notebook```

Since multiple characters delimiter is not supported, I had to transform data:
<pre>
zcat users.dat.gz | perl -nle '@arr=split/::/; print join("\t", @arr)' | gzip -c > users.data.tsv.gz
zcat movies.dat.gz | perl -nle '@arr=split/::/; print join("\t", @arr)' | native2ascii -encoding ISO-8859-1 | native2ascii -encoding utf8 -reverse |  gzip -c > movies.dat.tsv.gz
zcat ratings.dat.gz | perl -nle '@arr=split/::/; print join("\t", @arr)' | gzip -c > ratings.dat.tsv.gz 
</pre>

In [1]:
import os
account=os.getenv('SF_ACCOUNT')
user=os.getenv('SF_USER')
password=os.getenv('SF_PASSWORD')

## Create Tables and Load Data

In [2]:
from snowflake.connector import connect
con = connect(
    account=account,
    user=user,
    password=password,
    database='testdb',
    schema='public',
)
cur = con.cursor()

In [3]:
cur.execute("""
CREATE OR REPLACE SCHEMA movielense_1m
""")

In [4]:
cur.execute("""
CREATE OR REPLACE TABLE users(
    user_id    integer,
    gender     varchar(1),
    age        integer,
    occupation integer,
    zip        string)""")
cur.execute("""
CREATE OR REPLACE TABLE movies(
    movie_id   integer,
    title      string,
    genres     string)""")
cur.execute("""
CREATE OR REPLACE TABLE ratings(
    user_id    integer,
    movie_id   integer,
    rating     integer,
    ts         integer)""")


In [5]:
aws_key=os.getenv("AWS_ACCESS_KEY_ID")
aws_secret = os.getenv("AWS_SECRET_ACCESS_KEY")
sf_s3_bucket=os.getenv("SF_S3_BUCKET")

In [6]:
for tbl in ['users', 'ratings', 'movies']:
    cur.execute("""
copy into {tbl} from s3://{s3_bucket}/movielense/1m/{tbl}.dat.tsv.gz
    credentials=(
        aws_key_id='{aws_key_id}' 
        aws_secret_key='{aws_secret_key}'
    )
    file_format= (
        field_delimiter='\t'
        type='csv'
    )
""".format(
        s3_bucket=sf_s3_bucket,
        tbl=tbl,
        aws_key_id=aws_key,
        aws_secret_key=aws_secret))

In [7]:
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
engine = create_engine(URL(
    account=account,
    user=user,
    password=password,
    database='testdb',
    schema='movielense_1m',
    ))

In [8]:
import pandas as pd

In [9]:
users = pd.read_sql_query("SELECT * FROM users", engine)

In [10]:
ratings = pd.read_sql_query("SELECT * FROM ratings", engine)

In [11]:
movies = pd.read_sql_query("SELECT * FROM movies", engine)

In [12]:
users[:5]

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [13]:
ratings[:5]

,user_id,movie_id,rating,ts
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [14]:
movies[:5]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
data = pd.merge(pd.merge(ratings, users), movies)

In [16]:
data

,user_id,movie_id,rating,ts,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama
5,18,1193,4,978156168,F,18,3,95825,One Flew Over the Cuckoo's Nest (1975),Drama
6,19,1193,5,982730936,M,1,10,48073,One Flew Over the Cuckoo's Nest (1975),Drama
7,24,1193,5,978136709,F,25,7,10023,One Flew Over the Cuckoo's Nest (1975),Drama
8,28,1193,3,978125194,F,25,1,14607,One Flew Over the Cuckoo's Nest (1975),Drama
9,33,1193,5,978557765,M,45,3,55421,One Flew Over the Cuckoo's Nest (1975),Drama


In [17]:
data.ix[0]

user_id                                            1
movie_id                                        1193
rating                                             5
ts                                         978300760
gender                                             F
age                                                1
occupation                                        10
zip                                            48067
title         One Flew Over the Cuckoo's Nest (1975)
genres                                         Drama
Name: 0, dtype: object

In [18]:
mean_ratings = pd.pivot_table(data, values='rating', index='title', columns='gender', aggfunc='mean')

In [19]:
mean_ratings[:5]

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024


In [20]:
ratings_by_title = data.groupby('title').size()

In [21]:
ratings_by_title[:10]

title
$1,000,000 Duck (1971)                37
'Night Mother (1986)                  70
'Til There Was You (1997)             52
'burbs, The (1989)                   303
...And Justice for All (1979)        199
1-900 (1994)                           2
10 Things I Hate About You (1999)    700
101 Dalmatians (1961)                565
101 Dalmatians (1996)                364
12 Angry Men (1957)                  616
dtype: int64

In [22]:
active_titles = ratings_by_title.index[ratings_by_title>=250]

In [23]:
active_titles

Index([''burbs, The (1989)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1961)', '101 Dalmatians (1996)', '12 Angry Men (1957)',
       '13th Warrior, The (1999)', '2 Days in the Valley (1996)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '2010 (1984)',
       ...
       'X-Men (2000)', 'Year of Living Dangerously (1982)',
       'Yellow Submarine (1968)', 'You've Got Mail (1998)',
       'Young Frankenstein (1974)', 'Young Guns (1988)',
       'Young Guns II (1990)', 'Young Sherlock Holmes (1985)',
       'Zero Effect (1998)', 'eXistenZ (1999)'],
      dtype='object', name='title', length=1216)

In [24]:
mean_ratings = mean_ratings.ix[active_titles]

In [25]:
mean_ratings

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421
"13th Warrior, The (1999)",3.112000,3.168000
2 Days in the Valley (1996),3.488889,3.244813
"20,000 Leagues Under the Sea (1954)",3.670103,3.709205
2001: A Space Odyssey (1968),3.825581,4.129738


In [26]:
top_female_ratings = mean_ratings.sort_values(by='F', ascending=False)

In [27]:
top_female_ratings[:10]

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415
"Shawshank Redemption, The (1994)",4.539075,4.560625
"Grand Day Out, A (1992)",4.537879,4.293255
To Kill a Mockingbird (1962),4.536667,4.372611
Creature Comforts (1990),4.513889,4.272277


## Measuring rating disagreement

In [28]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']

In [29]:
sorted_by_diff = mean_ratings.sort_values(by='diff')

In [30]:
sorted_by_diff[:15]

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777
Anastasia (1997),3.800000,3.281609,-0.518391
"Rocky Horror Picture Show, The (1975)",3.673016,3.160131,-0.512885
"Color Purple, The (1985)",4.158192,3.659341,-0.498851
"Age of Innocence, The (1993)",3.827068,3.339506,-0.487561


In [31]:
sorted_by_diff[::-1][:15]

gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
Evil Dead II (Dead By Dawn) (1987),3.297297,3.909283,0.611985
"Hidden, The (1987)",3.137931,3.745098,0.607167
Rocky III (1982),2.361702,2.943503,0.581801
Caddyshack (1980),3.396135,3.969737,0.573602


In [32]:
rating_std_by_title = data.groupby('title')['rating'].std()

In [33]:
rating_std_by_title = rating_std_by_title.ix[active_titles]

In [34]:
rating_std_by_title.sort_values(ascending=False)[:10]

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
Eyes Wide Shut (1999)                    1.259624
Evita (1996)                             1.253631
Billy Madison (1995)                     1.249970
Fear and Loathing in Las Vegas (1998)    1.246408
Bicentennial Man (1999)                  1.245533
Name: rating, dtype: float64